In [51]:
%load_ext autoreload
%autoreload 2
from datasets import DatasetDict
from pysentimiento.lince.ner import load_datasets


train_dataset, dev_dataset, test_dataset = load_datasets("es")

lince_ner = DatasetDict(
    train=train_dataset,
    validation=dev_dataset,
    test=test_dataset,
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Reusing dataset lince (/home/jmperez/.cache/huggingface/datasets/lince/ner_spaeng/1.0.0/10d41747f55f0849fa84ac579ea1acfa7df49aa2015b60426bc459c111b3d589)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /home/jmperez/.cache/huggingface/datasets/lince/ner_spaeng/1.0.0/10d41747f55f0849fa84ac579ea1acfa7df49aa2015b60426bc459c111b3d589/cache-96b92c9276007a0d.arrow
Loading cached processed dataset at /home/jmperez/.cache/huggingface/datasets/lince/ner_spaeng/1.0.0/10d41747f55f0849fa84ac579ea1acfa7df49aa2015b60426bc459c111b3d589/cache-5e10b10d764d61ca.arrow
Loading cached processed dataset at /home/jmperez/.cache/huggingface/datasets/lince/ner_spaeng/1.0.0/10d41747f55f0849fa84ac579ea1acfa7df49aa2015b60426bc459c111b3d589/cache-168c9ba86661591e.arrow
Loading cached processed dataset at /home/jmperez/.cache/huggingface/datasets/lince/ner_spaeng/1.0.0/10d41747f55f0849fa84ac579ea1acfa7df49aa2015b60426bc459c111b3d589/cache-0e043540ac780ce8.arrow
Loading cached processed dataset at /home/jmperez/.cache/huggingface/datasets/lince/ner_spaeng/1.0.0/10d41747f55f0849fa84ac579ea1acfa7df49aa2015b60426bc459c111b3d589/cache-95d82bb339ffc16c.arrow
Loading cached processed 

In [15]:
train_dataset[101]["words"]

['@usuario', 'JA', '!', 'yo', 'solo', 'hise', 'la', 'mono']

In [16]:
idx = 11065
print(lince_ner["train"][idx]["ner"])
print(lince_ner["train"][idx]["labels"])
lince_ner["train"][idx].keys(), lince_ner["train"][idx]

['O', 'O', 'O', 'B-OTHER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'O', 'B-PER', 'O', 'O', 'O', 'O', 'O']
[0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 11, 12, 0, 11, 0, 0, 0, 0, 0]


(dict_keys(['ner', 'words', 'lid', 'labels', 'idx']),
 {'ner': ['O',
   'O',
   'O',
   'B-OTHER',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'B-PER',
   'I-PER',
   'O',
   'B-PER',
   'O',
   'O',
   'O',
   'O',
   'O'],
  'words': ['Im',
   'stuck',
   'in',
   'bachata',
   'for',
   'the',
   'rest',
   'of',
   'the',
   'week',
   '...',
   'prince',
   'royce',
   'and',
   'romeo',
   'are',
   'killing',
   'it',
   '!',
   ':)'],
  'lid': ['lang1',
   'lang1',
   'lang1',
   'lang2',
   'lang1',
   'lang1',
   'lang1',
   'lang1',
   'lang1',
   'lang1',
   'other',
   'ne',
   'ne',
   'lang1',
   'ne',
   'lang1',
   'lang1',
   'lang1',
   'other',
   'other'],
  'labels': [0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 11, 12, 0, 11, 0, 0, 0, 0, 0],
  'idx': 11065})

In [33]:
import torch
from transformers import AutoModelForTokenClassification, AutoTokenizer


device = "cuda" if torch.cuda.is_available() else "cpu"
max_length = 128

model_name = "pysentimiento/robertuito-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.model_max_length = max_length

In [34]:

words = lince_ner["train"][idx+2]["words"]
labels = lince_ner["train"][idx+2]["labels"]
tokens = tokenizer(words, is_split_into_words=True)

inv_vocab = {v:k for k, v in tokenizer.get_vocab().items()}

print(words)
list(zip(tokens.word_ids(), [inv_vocab[t] for t in tokens["input_ids"]]))

['#mlrt', '>>>', 'has', 'me', 'dying', '😂👌']


[(None, '<s>'),
 (0, '▁#'),
 (0, 'm'),
 (0, 'l'),
 (0, 'rt'),
 (1, '▁'),
 (2, '▁has'),
 (3, '▁me'),
 (4, '▁dying'),
 (5, '▁'),
 (None, '</s>')]

In [35]:
labels, tokens.word_ids()

([0, 0, 0, 0, 0, 0], [None, 0, 0, 0, 0, 1, 2, 3, 4, 5, None])

In [36]:
from pysentimiento.lince.ner import align_labels_with_tokens, tokenize_and_align_labels

align_labels_with_tokens(labels, tokens.word_ids())


[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100]

In [50]:
examples = train_dataset.select(range(32))

    model, tokenizer = load_model(
        base_model, label2id=label2id, id2label=id2label,
        max_length=max_length,
        auto_class=AutoModelForTokenClassification
    )

    tokenized_datasets = lince_ner.map(
        lambda x: tokenize_and_align_labels(x, tokenizer),
        batched=True,
        remove_columns=lince_ner["train"].column_names,
    )
ret = tokenize_and_align_labels(examples, tokenizer)

idx = 1
list(zip(ret[idx].tokens, ret["labels"][idx]))

[('<s>', -100),
 ('▁why', 0),
 ('▁does', 0),
 ('▁n', 0),
 ("'t", 0),
 ('▁louis', 7),
 ('▁v', 8),
 ('uit', 8),
 ('ton', 8),
 ('▁make', 0),
 ('▁men', 0),
 ("▁'s", 0),
 ('▁wat', 0),
 ('ches', 0),
 ('▁', 0),
 ('</s>', -100)]

In [52]:
from pysentimiento.training import load_model
from pysentimiento.lince.ner import label2id, id2label

base_model = "pysentimiento/robertuito-base-uncased"

model, tokenizer = load_model(
    base_model, label2id=label2id, id2label=id2label,
    max_length=max_length,
    auto_class=AutoModelForTokenClassification
)

tokenized_datasets = lince_ner.map(
    lambda x: tokenize_and_align_labels(x, tokenizer),
    batched=True,
    remove_columns=lince_ner["train"].column_names,
)

Loading model pysentimiento/robertuito-base-uncased


Downloading:   0%|          | 0.00/677 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/415M [00:00<?, ?B/s]

Some weights of the model checkpoint at pysentimiento/robertuito-base-uncased were not used when initializing RobertaForTokenClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at pysentimiento/robertuito-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a dow

  0%|          | 0/34 [00:00<?, ?ba/s]

  0%|          | 0/11 [00:00<?, ?ba/s]

  0%|          | 0/24 [00:00<?, ?ba/s]

In [53]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)



In [58]:
from datasets import load_metric

metric = load_metric("seqeval")

example = lince_ner["train"][1]

labels = example["ner"]
preds = labels.copy()
preds[0] = "B-LOC"

metric.compute(predictions=[preds], references=[labels])

/home/jmperez/.cache/pypoetry/virtualenvs/pysentimiento-bwlKzHxB-py3.7/lib/python3.7/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'LOC': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 0},
 'ORG': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 0.5,
 'overall_recall': 1.0,
 'overall_f1': 0.6666666666666666,
 'overall_accuracy': 0.9}

In [ ]:
import numpy as np
from seqeval.metrics import f1_score


def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[id2label[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    ret = {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "macro_f1": all_metrics["overall_f1"],
        "micro_f1": f1_score(true_labels, true_predictions, average="micro"),
        "accuracy": all_metrics["overall_accuracy"],
    }

    for k, v in all_metrics.items():
        if not k.startswith("overall"):
            ret[k + "_f1"] = v["f1"]
            ret[k + "_precision"] = v["precision"]
            ret[k + "_recall"] = v["recall"]

    return ret 
